# Free swelling using geometry

In [104]:
import numpy as np
import matplotlib.pyplot as plt

from ngsolve import *
from netgen.csg import *
from ngsolve.webgui import Draw

In [105]:
# Geometry of 1/8 
L = 90
d = 23.5
L3 = 3.0

left  = Plane (Pnt(0,0,0), Vec(-1,0,0) ).bc('left')
right = Plane (Pnt(0.5*L,0,0), Vec( 1,0,0) )
bot = Plane (Pnt(0,0,0), Vec(0,-1,0) ).bc('bot')
top  = Plane (Pnt(0,0.5*d,0), Vec(0, 1,0) )
back   = Plane (Pnt(0,0,0), Vec(0,0,-1) ).bc('back')
front   = Plane (Pnt(0,0,0.5*L3), Vec(0,0, 1) )

brick = left * right * front * back * bot * top

geo = CSGeometry()
geo.Add (brick)
mesh = Mesh(geo.GenerateMesh(maxh=1))
Draw(mesh)
print(mesh.GetBoundaries())

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

('left', 'bot', 'default', 'default', 'back', 'default')


# Parameters

In [106]:
global phi0
#phi0 = 0.2035 # or 0.3
phi0 = 0.3

global chi 
chi = 0.45

global entropic_unit
entropic_unit=136.6   #This is (k_B * T)/(V_m), measured in Megapascals

global G    #It is the shear modulus. Will be defined in the next script (next jupy cell).

phi = lambda J: phi0/J
dH = lambda J: (1-1/1000) * phi(J) + np.log(1-phi(J)) + chi * phi(J)**2

#gammafun = lambda lamb: -dH(lamb)/lamb # bonded
gammafun = lambda lamb: -dH(lamb**3)*lamb

lambda_target = 1.49
gamma_target = gammafun(lambda_target)
G_target = gamma_target*entropic_unit
print("gamma target:", gamma_target)
print("shear modulus target:", G_target)

print("lo que tiene que crecer:", d*(lambda_target-1))
G=G_target
print( "desplazamineto esperados:", (lambda_target-1)/2*np.array([L,d,L3]) )
#v = np.linspace(1,2,100)
#plt.plot(v,gammafun(v)*entropic_unit)

gamma target: 0.001145522071527073
shear modulus target: 0.15647831497059816
lo que tiene que crecer: 11.515
desplazamineto esperados: [22.05    5.7575  0.735 ]


In [107]:
def F(u):
    return Id(mesh.dim) + Grad(u)

global gamma
gamma = G/entropic_unit
print(gamma)

def Gels (F):
    J = Det(F)
    phi = phi0/J
    H = (J - phi0)*log(1-phi)  + phi0 * chi*(1-phi) + phi0/1000*log(phi)
    C = F.trans * F
    return 0.5*gamma*(Trace(C)) + H

0.001145522071527073


In [108]:
# Finite element space with slipp boundary conditions on back|bot|left
V = VectorH1(mesh, order=2, dirichletx = 'left', dirichlety='bot', dirichletz='back')

# Construction of bilinear form
u  = V.TrialFunction()
I = Id(mesh.dim)
F = I + Grad(u)
a = BilinearForm(V)

# hydrogel model
a += Variation(  Gels (F).Compile() * dx)

In [156]:
def Norm(vec):
    return sqrt(abs(InnerProduct(vec,vec)))

def backward_sub(U,x):
    n = U.shape[0]
    x = np.zeros(n)
    for i in range(n-1,-1,-1):
        x[i] = (U[i,-1] - np.dot(U[i,:-1],x))/U[i,i]
    return x
def SolveNonlinearMinProblem(a,gfu,tol=1e-08,maxits=250, m = 5, scenes=None):
    res = gfu.vec.CreateVector()
    du  = gfu.vec.CreateVector()
    g = [res.FV().NumPy()[:]] # initialize with 0 
    ### first iteration by hand
    a.Apply (gfu.vec, res)
    a.AssembleLinearization (gfu.vec)
    inv = a.mat.Inverse(V.FreeDofs())
    # x =x - D^-1 F * F
    du.data = 0.5*inv * res

    # gfu starts at 0!!

    # check if need to transpose
    x = np.array([gfu.vec.FV().NumPy()[:], -du.FV().NumPy()[:]]).T
    print(x.shape)
    # vector of residuals
    g.append(res.FV().NumPy()[:])
    g = np.array(g).T
    # matrix of increments
    G_k = np.array([g[:,1]-g[:,0]]).T
    
    
    X_k = np.array([x[1]-x[0]])

    k = 2
    
    while k < maxits:
        m_k = min(k,m)
        Q , R = np.linalg.qr(G_k)

        # compute gamma
        if R.shape[1] ==1:

            print(G_k)
            print(R)
            gamma_k = Q.T@g[:,k-1] / R[0,0]

        elif Q.shape[1] == 1:

            gamma_k = backward_sub(R,Q[0,0] * g[k-1])
        
        # compute next iteration
        print ("Newton iteration {:3}".format(k),end=", ")
        print ("energy = {:16}".format(a.Energy(gfu.vec)),end="")
        #solve linearized problem:
        a.Apply (gfu.vec, res)
        a.AssembleLinearization (gfu.vec)
        inv = a.mat.Inverse(V.FreeDofs())
        alpha = 5e-1
        du.data = alpha * inv * res
        if gamma_k.shape[0] == 1:
            iter = x[k-1] + g[-1] - (X_k + G_k)*gamma_k
            x = np.c_[x,iter[0]]
        else:
            iter = x[k-1] + g[-1] - (X_k + G_k)@gamma_k
            x = np.c_[x,iter[0]]

        g = np.c_[g, res.FV().NumPy()[:]]
        print(g.shape)
        print(G_k.shape)
        G_k =np.c_[G_k, g[:,-1] - g[:,-2]]
        X_k = np.c_[X_k, x[:,k] - x[:,k-1]]

        #update iteration<
        gfu.vec.FV().NumPy()[:] = x[k]
        n = len(X_k)
        if n> m_k:
            X_k = X_k[-m_k:]
            G_k = G_k[-m_k:]
        k+=1
        #stopping criteria
        stopcritval = sqrt(abs(InnerProduct(du,res)))
        print ("<A u",k,", A u",k,">_{-1}^0.5 = ", stopcritval)
        if stopcritval < tol:
            break
    return gfu

In [157]:
# Solve

gfu = GridFunction(V)
gfu.vec[:] = 0

# scene = Draw (gfu, mesh, "u", deformation=True)

# scene0 = Draw (gfu.components[0], mesh, "u", deformation=True)
# scene1 = Draw (gfu.components[1], mesh, "u", deformation=True)

# scene2 = Draw (gfu.components[2], mesh, "u", deformation=True)

# SetVisualization (deformation=True)

res = gfu.vec.CreateVector()
du = gfu.vec.CreateVector()

# scenes = [scene, scene0, scene1, scene2]
# scenes = [scene]
gfu = SolveNonlinearMinProblem(a,gfu, scenes=scenes, maxits=10)

(22503, 2)
[[0.]
 [0.]
 [0.]
 ...
 [0.]
 [0.]
 [0.]]
[[0.]]
Newton iteration   2, energy = -121.99481492764417

C:\Users\sfsta\AppData\Local\Temp\ipykernel_23240\2618068739.py:45: RuntimeWarning: divide by zero encountered in divide
  gamma_k = Q.T@g[:,k-1] / R[0,0]
C:\Users\sfsta\AppData\Local\Temp\ipykernel_23240\2618068739.py:61: RuntimeWarning: invalid value encountered in multiply
  iter = x[k-1] + g[-1] - (X_k + G_k)*gamma_k


ValueError: all the input array dimensions except for the concatenation axis must match exactly, but along dimension 0, the array at index 0 has size 22503 and the array at index 1 has size 2

In [ ]:
Draw (gfu, mesh, "u", deformation=True)

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

BaseWebGuiScene

In [ ]:
from RUN_Gel_free_swelling import gel_free_swelling, Solve_gel_free_swelling
GEL = gel_free_swelling()
GEL.geometry()

modelling = Solve_gel_free_swelling(GEL)
modelling.Solve()
Draw(modelling.gfu)

ModuleNotFoundError: No module named 'RUN_Gel_free_swelling'